In [ ]:
# Reading data from data file
import panda as pd
data_as = pd.read_csv("data_airspace.csv")


In [ ]:
# Extracting trajectory data from columns of the data file to corresponding variables

long_1 = data_as.loc[:,"longitude_1"]
lat_1 = data_as.loc[:,"latitude_1"]
alt_1 = data_as.loc[:,"altitude_1"]

long_2 = data_as.loc[:,"longitude_2"]
lat_2 = data_as.loc[:,"latitude_2"]
alt_2 = data_as.loc[:,"altitude_2"]

u_1 = data_as.loc[:,"velocity_u_1"]
v_1 = data_as.loc[:,"velocity_v_1"]
w_1 = data_as.loc[:,"velocity_w_1"]

u_2 = data_as.loc[:,"velocity_u_2"]
v_2 = data_as.loc[:,"velocity_v_2"]
w_2 = data_as.loc[:,"velocity_w_2"]


In [ ]:
# Extracting sector data from columns of the data file to corresponding variables

x_sector = data_as.loc[:,"coordinates_x"]
y_sector = data_as.loc[:,"coordinates_y"]
z_sector = data_as.loc[:,"floor_to_ceiling_height"]


In [ ]:
import numpy as np

# 1_Number of A/C
# we should enumerate number of columns / 6
No_A/C = data_as.columns

In [ ]:
# 3_Area and volume of the sector
#  Shoelace formula because it is faster
def PolyArea(x,y):
    return 0.5*np.abs(np.dot(x,np.roll(y,1))-np.dot(y,np.roll(x,1)))

area = PolyArea(x_sector,y_sector)
volume = area*z_sector

In [ ]:
# 4_A/C density: #A/C / Area of the sector

density = No_A/C/volume


In [ ]:
# 5_ number of A/C with Heading Change greater than 15º
# Achtung: zero in the denominator of slope[i]
from math import pi

def Heading_change(long,lat):
    for i in range(1,(len(long)-1)):
        delta_long[i]=long[i+1]-long[i]
        delta_lat[i]=lat[i+1]-lat[i]
        slope[i]=delta_lat[i]/delta_long[i]
        arctan[i]=np.arctan( slope[i] )*180/pi
        logical_atan[i]=arctan[i]>15

    if '1' in logical_atan
        return 1
    else:
        return 0

count_Headin_Change = Heading_change(long_1,lat_1) + Heading_change(long_2,lat_2)


In [ ]:
# 6_ number of A/C with Altitude Change greater than 750 feet (i.e., 228.6 meter)

def Altitude_Change(alt):
    delta_alt[i]=alt[i+1]-alt[i]
    logical_delta_alt[i]=delta_alt[i]>228.6
    if '1' in logical_delta_alt:
        return 1
    else:
        return 0

count_Altitude_Change =  Altitude_Change(alt_1) + Altitude_Change(alt_2)

In [ ]:
# 7_ number of A/C with 3D Euclidean distance between 0-5 nautical miles (i.e., 0-9260 meters)

def distance(long1,lat1,alt1, long2,lat2,alt2):


diff_long = long1(1,i)-long2
diff_lat(i)=lat(1,i)-lat(2,i);
diff_alt(i)=lat(1,i)-lat(2,i);
dist(i)=(diff_lat(i)^2 + diff_long(i)^2 + diff_alt(i)^2)^0.5;
logical_dist(i)=dist(i)<5;
end

counter=0;
    if sum(logical_dist)>0
        counter=counter+1;
    end 
disp ("#A/C with 3-D Euclidean distance between 0-5 nautical miles:")
disp (counter)
disp ("pair")


In [ ]:
# Task load number 1: average_transit_time

def average_tt(long)
    return len(long)



In [23]:
len(c)

5

In [26]:
w=b>1
w+w*3

4

In [30]:
q=[5,6,7,8,9]

q+a
a+q

[1, 2, 3, 5, 6, 7, 8, 9]

hello